In [1]:
import tensorflow as tf
from gym_trading_env.environments import TradingEnv
import gymnasium as gym
from datetime import datetime
import pandas as pd
import numpy as np
import gym_trading_env
from time import time

from data_provider import MarketDataProvider
from agent import TradingAgent

In [2]:
data_provider = MarketDataProvider(exchange_name="binance", symbol="BTC/USDT", timeframe="1h", data_directory="data", since=datetime(year= 2020, month= 1, day=1))

In [3]:
df = data_provider.fetch_data()

BTC/USDT downloaded from binance and stored at data/binance-BTCUSDT-1h.pkl


In [4]:
df

,open,high,low,close,volume,date_close,feature_open,feature_high,feature_low,feature_close,...,feature_senkou_a,feature_senkou_b,feature_kijun,feature_tenkan,feature_natural_market_river,feature_natural_market_mirror,feature_natural_market_mirror_nma,feature_natural_market_river_nma,feature_natural_market_mirror_nma_diff,feature_natural_market_river_nma_diff
date_open,,,,,,,,,,,,,,,,,,,,,
2020-01-02 01:00:00,-1.379832,-1.380831,-1.380500,-1.381221,-0.722366,2020-01-02 02:00:00,-1.379832,-1.380831,-1.380500,-1.381221,...,-1.381976,-1.381367,-1.382080,-1.380542,0.382316,0.367799,0.072401,0.091883,0.225089,0.197670
2020-01-02 02:00:00,-1.381141,-1.382139,-1.382455,-1.382655,-0.533450,2020-01-02 03:00:00,-1.381141,-1.382139,-1.382455,-1.382655,...,-1.382677,-1.382017,-1.382728,-1.381296,0.557765,0.552542,0.121774,0.124406,0.267940,0.142990
2020-01-02 03:00:00,-1.382573,-1.383563,-1.384753,-1.385228,-0.500723,2020-01-02 04:00:00,-1.382573,-1.383563,-1.384753,-1.385228,...,-1.383883,-1.383164,-1.383873,-1.382562,0.937581,0.895794,0.256482,0.238613,0.730572,0.501521
2020-01-02 04:00:00,-1.385171,-1.384086,-1.384994,-1.384405,-0.559336,2020-01-02 05:00:00,-1.385171,-1.384086,-1.384994,-1.384405,...,-1.384003,-1.383284,-1.383994,-1.382682,0.363799,0.576909,0.307409,0.266745,0.276365,0.123717
2020-01-02 05:00:00,-1.384299,-1.384666,-1.384322,-1.384619,-0.690357,2020-01-02 06:00:00,-1.384299,-1.384666,-1.384322,-1.384619,...,-1.384007,-1.383284,-1.383994,-1.382689,0.359905,0.505342,0.315093,0.266906,0.041929,0.000941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 04:00:00,-0.072742,-0.078991,-0.066100,-0.071788,-0.667682,2023-10-16 05:00:00,-0.072742,-0.078991,-0.066100,-0.071788,...,-0.080347,-0.081124,-0.082986,-0.076969,-0.205095,-0.240132,-0.480506,-0.441213,0.005200,0.006055
2023-10-16 05:00:00,-0.071746,-0.033247,-0.062343,-0.028363,0.822989,2023-10-16 06:00:00,-0.071746,-0.033247,-0.062343,-0.028363,...,-0.057555,-0.058607,-0.060500,-0.053883,-2.763878,-2.167767,-1.642148,-1.951608,-6.297453,-6.629341
2023-10-16 06:00:00,-0.028323,-0.033809,-0.027012,-0.035141,-0.149657,2023-10-16 07:00:00,-0.028323,-0.033809,-0.027012,-0.035141,...,-0.057555,-0.058607,-0.060500,-0.053883,-1.027419,-1.411724,-1.702571,-1.861384,-0.327310,0.396256


In [5]:
# discount factor
gamma = .99
# update frequency between online model and target model
tau =100
# Adam learning reate
learning_rate=0.0001
 # L2 regularization using norm 2 euclidian distance
l2_reg = 1e-6
# size of the prioritized replay buffer
replay_capacity = int(1e6)
# batch size to fetch from replay buffer
batch_size=4096
# epsilon greedy policy parameters 
epsilon_start = 1.0
epsilon_end = .01
epsilon_decay_steps = 250
epsilon_exponential_decay = .99

In [6]:
tf.keras.backend.clear_session()

In [7]:
risk_free_rate = 0.03
def reward_function(history):
    # Calculate portfolio daily returns
    portfolio_values = history["portfolio_valuation"]
    daily_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    
    # Calculate excess daily returns over risk-free rate
    excess_returns = daily_returns - risk_free_rate
    
    # Calculate the mean and standard deviation of excess returns
    mean_excess_return = np.mean(excess_returns)
    std_dev_excess_return = np.std(excess_returns)
    
    # Calculate Sharpe ratio
    if std_dev_excess_return != 0:
        sharpe_ratio = mean_excess_return / std_dev_excess_return
    else:
        sharpe_ratio = 0
    
    return sharpe_ratio

In [8]:
env = gym.make(
        "TradingEnv",
        name= "BTCUSD",
        df = df,
        windows= 5,
        positions = [ -1, -0.5, 0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 'random', #Initial position
        trading_fees = 0.01/100, # 0.01% per stock buy / sell
        borrow_interest_rate= 0.0003/100, #per timestep (= 1h here)
        reward_function = reward_function,
        portfolio_initial_value = 1000, # in FIAT (here, USD)
        max_episode_duration = 500,
        max_episode_steps=500
    )

In [9]:
state_dimensions = 85
num_actions=env.action_space.n

In [10]:
trading_agent = TradingAgent(state_dimensions=state_dimensions,
                 num_actions=num_actions,
                 learning_rate=learning_rate,
                 gamma=gamma,
                 epsilon_start=epsilon_start,
                 epsilon_end=epsilon_end,
                 epsilon_decay_steps=epsilon_decay_steps,
                 epsilon_exponential_decay=epsilon_exponential_decay,
                 replay_capacity=replay_capacity,
                 l2_reg=l2_reg,
                 tau=tau,
                 batch_size=batch_size,)

In [11]:
trading_agent.online_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 265)               22790     
                                                                 
 Dense_2 (Dense)             (None, 265)               70490     
                                                                 
 dropout (Dropout)           (None, 265)               0         
                                                                 
 Output (Dense)              (None, 7)                 1862      
                                                                 
Total params: 95142 (371.65 KB)
Trainable params: 95142 (371.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
total_steps = 0
max_episodes = 1000

In [13]:
start = time()
results = []
for episode in range(1, max_episodes + 1):
    this_state = env.reset()[0].reshape(1,-1)
    for episode_step in range(env.spec.max_episode_steps):
        action = trading_agent.epsilon_greedy_policy(this_state.reshape(1, -1))
        next_state, reward, done, truncated, info = env.step(action)

        trading_agent.memorize_transition(this_state, 
                                 action, 
                                 reward, 
                                 next_state.reshape(1,-1), 
                                 0.0 if done | truncated else 1.0)
        if trading_agent.train:
            trading_agent.experience_replay()
        if done:
            env.unwrapped.save_for_render(dir="render_logs")
            break
        this_state = next_state.reshape(1,-1)

    # # get DataFrame with seqence of actions, returns and nav values
    # result = env.unwrapped.simulator.result()
    
    # # get results of last step
    # final = result.iloc[-1]

    # # apply return (net of cost) of last action to last starting nav 
    # nav = final.nav * (1 + final.strategy_return)
    # navs.append(nav)

    # # market nav 
    # market_nav = final.market_nav
    # market_navs.append(market_nav)

    # # track difference between agent an market NAV results
    # diff = nav - market_nav
    # diffs.append(diff)
    
    # if episode % 10 == 0:
    #     track_results(episode, 
    #                   # show mov. average results for 100 (10) periods
    #                   np.mean(navs[-100:]), 
    #                   np.mean(navs[-10:]), 
    #                   np.mean(market_navs[-100:]), 
    #                   np.mean(market_navs[-10:]), 
    #                   # share of agent wins, defined as higher ending nav
    #                   np.sum([s > 0 for s in diffs[-100:]])/min(len(diffs), 100), 
    #                   time() - start, trading_agent.epsilon)
    # if len(diffs) > 25 and all([r > 0 for r in diffs[-25:]]):
    #     print(result.tail())
    #     break

env.close()

Market Return : 45.91%   |   Portfolio Return : 27.00%   |   
Market Return : 46.17%   |   Portfolio Return : 26.74%   |   
1/1 [==============================] - 0s 28ms/step
Market Return : -85.83%   |   Portfolio Return : -33.56%   |   
Market Return : -85.04%   |   Portfolio Return : -29.82%   |   
1/1 [==============================] - 0s 32ms/step
Market Return : -104.10%   |   Portfolio Return : -359.12%   |   
Market Return : -62.11%   |   Portfolio Return : -105.48%   |   
1/1 [==============================] - 0s 36ms/step
Market Return : -2.51%   |   Portfolio Return : -2.94%   |   
Market Return : -2.72%   |   Portfolio Return : -3.13%   |   
1/1 [==============================] - 0s 28ms/step
Market Return : 60.18%   |   Portfolio Return : 40.97%   |   
Market Return : 61.45%   |   Portfolio Return : 41.51%   |   
1/1 [==============================] - 0s 30ms/step
Market Return : -1.00%   |   Portfolio Return :  9.83%   |   
1/1 [==============================] - 0s 40ms/

KeyboardInterrupt: 